# MVP Final — Análisis de Sentimiento (Ternario) en Español  
### TF‑IDF + Regresión Logística (Calibrada) + Artefacto `joblib` listo para Back‑End

**Objetivo (Hackathon):** entregar un **modelo productivo** que reciba texto y prediga **Negativo / Neutro / Positivo**, devolviendo además una **probabilidad (0–1)** y una bandera `review_required` para revisión humana cuando la confianza sea baja.

**Label:** derivado de `stars` → **1–2 Negativo, 3 Neutro, 4–5 Positivo**  
**Alcance:** se filtra a español (`language == "es"`)

> Este notebook está pensado para jurado: reproducible, claro, con métricas, validación de input, artefacto serializado y contrato de integración.

## 0) Setup y dependencias

**Requisitos (pip):**
- pandas, numpy, scikit-learn, joblib

> En Colab: `!pip -q install scikit-learn joblib pandas numpy`

In [1]:
# =========================
# 0) Setup y configuración
# =========================
import os
import re
import json
import time
import random
import hashlib
from dataclasses import dataclass
from typing import Any, Dict, Optional, Tuple

import numpy as np
import pandas as pd
import joblib

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    f1_score, precision_recall_fscore_support
)

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

pd.set_option("display.max_colwidth", 180)

## 1) Carga de datos

**Schema mínimo esperado:**
- `language` (ej. `"es"`)
- `stars` (1–5)
- `review_title`, `review_body`

Ajusta los paths según tu entorno.

In [3]:
# =========================
# 1) Carga de datos
# =========================
TRAIN_PATH = "train.csv"
VAL_PATH   = "validation.csv"
TEST_PATH  = "test.csv"

train = pd.read_csv(TRAIN_PATH)
val   = pd.read_csv(VAL_PATH)
test  = pd.read_csv(TEST_PATH)

# Limpieza de columnas índice típicas
for df in (train, val, test):
    for c in ("Unnamed: 0", "index"):
        if c in df.columns:
            df.drop(columns=c, inplace=True)

print("Shapes:", train.shape, val.shape, test.shape)
train.head(2)

Shapes: (1200000, 8) (30000, 8) (30000, 8)


,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,de_0203609,product_de_0865382,reviewer_de_0267719,1,Armband ist leider nach 1 Jahr kaputt gegangen,Leider nach 1 Jahr kaputt,de,sports
1,de_0559494,product_de_0678997,reviewer_de_0783625,1,In der Lieferung war nur Ein Akku!,EINS statt ZWEI Akkus!!!,de,home_improvement


## 2) EDA rápida (orientada a producción)

Buscamos:
- Distribución de idiomas
- Distribución de estrellas
- Consistencia de columnas

In [4]:
def size_report(df: pd.DataFrame, name: str) -> None:
    print(f"\n{name}")
    print("Total registros:", len(df))
    if "language" in df.columns:
        print("\nIdiomas (top):")
        print(df["language"].value_counts().head(10))
    if "stars" in df.columns:
        print("\nStars:")
        print(df["stars"].value_counts().sort_index())

size_report(train, "TRAIN")
size_report(val, "VALIDATION")
size_report(test, "TEST")


TRAIN
Total registros: 1200000

Idiomas (top):
language
de    200000
en    200000
es    200000
fr    200000
ja    200000
zh    200000
Name: count, dtype: int64

Stars:
stars
1    240000
2    240000
3    240000
4    240000
5    240000
Name: count, dtype: int64

VALIDATION
Total registros: 30000

Idiomas (top):
language
de    5000
en    5000
es    5000
fr    5000
ja    5000
zh    5000
Name: count, dtype: int64

Stars:
stars
1    6000
2    6000
3    6000
4    6000
5    6000
Name: count, dtype: int64

TEST
Total registros: 30000

Idiomas (top):
language
de    5000
en    5000
es    5000
fr    5000
ja    5000
zh    5000
Name: count, dtype: int64

Stars:
stars
1    6000
2    6000
3    6000
4    6000
5    6000
Name: count, dtype: int64


## 3) Preparación de texto y labels

**Decisiones productivas de limpieza (conservadora):**
- Minimizamos supuestos fuertes (sin lematización obligatoria)
- Lowercase, remoción de URLs/menciones/hashtags
- Mantener caracteres relevantes para español (áéíóúñü)
- Normalizar espacios
- Filtrar textos demasiado cortos

**Texto final:** concatenación de `review_title` + `review_body`.

In [5]:
# =========================
# 3) Preparación de texto y labels
# =========================

def build_text_raw(df: pd.DataFrame) -> pd.Series:
    return (
        df["review_title"].fillna("").astype(str).str.strip() + ". " +
        df["review_body"].fillna("").astype(str).str.strip()
    ).str.strip()

def stars_to_sentiment(stars: int) -> str:
    s = int(stars)
    if s in (1, 2):
        return "Negativo"
    if s == 3:
        return "Neutro"
    return "Positivo"

_noise_re = re.compile(r"(http\S+|www\.\S+|@\w+|#\w+)", re.IGNORECASE)
_space_re = re.compile(r"\s+")

def clean_text(text: str) -> str:
    if text is None:
        return ""
    t = str(text).lower()
    t = _noise_re.sub(" ", t)
    t = re.sub(r"[^\w\sáéíóúñü]", " ", t)   # conserva caracteres ES
    t = _space_re.sub(" ", t).strip()
    return t

def prepare_es(df: pd.DataFrame, min_len: int = 5) -> pd.DataFrame:
    # filtro por idioma
    df_es = df[df["language"] == "es"].copy()
    df_es["text_raw"] = build_text_raw(df_es)
    df_es["text_clean"] = df_es["text_raw"].map(clean_text)
    df_es["text_len"] = df_es["text_clean"].str.len()
    df_es = df_es[df_es["text_len"] >= min_len].copy()
    df_es["sentiment"] = df_es["stars"].astype(int).map(stars_to_sentiment)
    return df_es[["text_raw", "text_clean", "sentiment"]]

train_es = prepare_es(train)
val_es   = prepare_es(val)
test_es  = prepare_es(test)

print("TRAIN (es):", len(train_es))
print("VAL (es):", len(val_es))
print("TEST (es):", len(test_es))

train_es.head(3)

TRAIN (es): 200000
VAL (es): 5000
TEST (es): 5000


,text_raw,text_clean,sentiment
400000,television Nevir. Nada bueno se me fue ka pantalla en menos de 8 meses y no he recibido respuesta del fabricante,television nevir nada bueno se me fue ka pantalla en menos de 8 meses y no he recibido respuesta del fabricante,Negativo
400001,"Dinero tirado a la basura con esta compra. Horrible, nos tuvimos que comprar otro porque ni nosotros que sabemos inglés, ni un informático, después de una hora fue capaz de ins...",dinero tirado a la basura con esta compra horrible nos tuvimos que comprar otro porque ni nosotros que sabemos inglés ni un informático después de una hora fue capaz de instalarlo,Negativo
400002,"solo llega una unidad cuando te obligan a comprar dos. Te obligan a comprar dos unidades y te llega solo una y no hay forma de reclamar, una autentica estafa, no compreis!!",solo llega una unidad cuando te obligan a comprar dos te obligan a comprar dos unidades y te llega solo una y no hay forma de reclamar una autentica estafa no compreis,Negativo


### Distribución de clases (para anticipar desbalance)

In [6]:
def sentiment_dist(df: pd.DataFrame, name: str) -> None:
    print(f"\n{name}")
    print(df["sentiment"].value_counts())
    print("\nProporciones:")
    print(df["sentiment"].value_counts(normalize=True).round(3))

sentiment_dist(train_es, "TRAIN (es)")
sentiment_dist(val_es, "VAL (es)")
sentiment_dist(test_es, "TEST (es)")


TRAIN (es)
sentiment
Negativo    80000
Positivo    80000
Neutro      40000
Name: count, dtype: int64

Proporciones:
sentiment
Negativo    0.4
Positivo    0.4
Neutro      0.2
Name: proportion, dtype: float64

VAL (es)
sentiment
Negativo    2000
Positivo    2000
Neutro      1000
Name: count, dtype: int64

Proporciones:
sentiment
Negativo    0.4
Positivo    0.4
Neutro      0.2
Name: proportion, dtype: float64

TEST (es)
sentiment
Negativo    2000
Positivo    2000
Neutro      1000
Name: count, dtype: int64

Proporciones:
sentiment
Negativo    0.4
Positivo    0.4
Neutro      0.2
Name: proportion, dtype: float64


## 4) Baseline rápido (TF‑IDF + Logistic Regression) — sanity check

Esto valida que el flujo completo funciona antes de optimizar.

In [7]:
X_train = train_es["text_raw"]
y_train = train_es["sentiment"]

X_val = val_es["text_raw"]
y_val = val_es["sentiment"]

baseline_pipe = Pipeline(steps=[
    ("cleaner", (lambda x: x)),  # placeholder para mostrar baseline mínimo
])

# Baseline real con cleaner + tfidf + LR
class TextCleaner(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return [clean_text(x) for x in X]

baseline = Pipeline(steps=[
    ("cleaner", TextCleaner()),
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.95, sublinear_tf=True)),
    ("model", LogisticRegression(
        solver="lbfgs", max_iter=2000, n_jobs=-1,
        random_state=SEED, class_weight="balanced"
    ))
])

baseline.fit(X_train, y_train)
pred_val = baseline.predict(X_val)

print("Baseline Accuracy (val):", round(float(accuracy_score(y_val, pred_val)), 4))
print("Baseline F1-macro (val):", round(float(f1_score(y_val, pred_val, average="macro")), 4))
print("\nClassification report (val):\n")
print(classification_report(y_val, pred_val))

Baseline Accuracy (val): 0.7536
Baseline F1-macro (val): 0.7183

Classification report (val):

              precision    recall  f1-score   support

    Negativo       0.83      0.79      0.81      2000
      Neutro       0.45      0.56      0.50      1000
    Positivo       0.89      0.82      0.85      2000

    accuracy                           0.75      5000
   macro avg       0.72      0.72      0.72      5000
weighted avg       0.78      0.75      0.76      5000



## 5) Optimización (GridSearchCV) con CV estratificada **sin leakage**

Para una nota 10/10:
- Hacemos selección de hiperparámetros con CV
- Todo dentro de `Pipeline` → evita filtraciones de vocabulario/TF‑IDF entre folds

**Métrica principal:** F1-macro (justa con desbalance).

In [8]:
# =========================
# 5) GridSearchCV (productivo)
# =========================
pipe = Pipeline(steps=[
    ("cleaner", TextCleaner()),
    ("tfidf", TfidfVectorizer(sublinear_tf=True, max_features=200000)),
    ("model", LogisticRegression(
        solver="lbfgs", max_iter=3000,
        random_state=SEED, class_weight="balanced"
    ))
])

param_grid = {
    "tfidf__ngram_range": [(1,2)],
    "tfidf__min_df": [3, 5],
    "tfidf__max_df": [0.95],
    "model__C": [1.0, 2.0]
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)

gs = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring="f1_macro",
    n_jobs=1,      # evita caída de workers por RAM
    cv=cv,
    verbose=1
)

gs.fit(X_train, y_train)

print("Best CV F1-macro:", round(float(gs.best_score_), 4))
print("Best params:", gs.best_params_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
Best CV F1-macro: 0.7143
Best params: {'model__C': 1.0, 'tfidf__max_df': 0.95, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2)}


## 6) Calibración de probabilidades (clave para API)

El backend necesita probabilidades interpretables.  
Usamos `CalibratedClassifierCV` (sigmoid) sobre el mejor modelo encontrado.

> Nota: calibrar mejora confiabilidad de `probabilidad` (no siempre sube F1, pero mejora calidad de confianza).

In [9]:
# =========================
# 6) Calibración
# =========================
best_pipe = gs.best_estimator_

# Extraer el clasificador LR y re-armar un pipeline con calibración
# (Se calibra SOLO el clasificador final, manteniendo el mismo preprocesamiento)
best_cleaner = best_pipe.named_steps["cleaner"]
best_tfidf   = best_pipe.named_steps["tfidf"]
best_lr      = best_pipe.named_steps["model"]

calibrated_clf = CalibratedClassifierCV(
    estimator=best_lr,
    method="sigmoid",
    cv=3
)

final_pipe = Pipeline(steps=[
    ("cleaner", best_cleaner),
    ("tfidf", best_tfidf),
    ("model", calibrated_clf)
])

final_pipe.fit(X_train, y_train)

# Evaluación rápida en VAL
val_pred = final_pipe.predict(X_val)
val_proba = final_pipe.predict_proba(X_val)
val_maxp = val_proba.max(axis=1)

print("Final Accuracy (val):", round(float(accuracy_score(y_val, val_pred)), 4))
print("Final F1-macro (val):", round(float(f1_score(y_val, val_pred, average='macro')), 4))
print("\nClassification report (val):\n")
print(classification_report(y_val, val_pred))

# Métrica de negocio típica: Recall de Negativo
labels = list(final_pipe.classes_)
neg_label = "Negativo" if "Negativo" in labels else labels[0]
print("Labels:", labels)

p_macro, r_macro, f_macro, _ = precision_recall_fscore_support(y_val, val_pred, average="macro")
print("\nMacro Precision/Recall/F1:", round(float(p_macro),4), round(float(r_macro),4), round(float(f_macro),4))

# Recall específico de Negativo
report = classification_report(y_val, val_pred, output_dict=True)
print("Recall Negativo (val):", round(float(report.get(neg_label, {}).get("recall", np.nan)), 4))

Final Accuracy (val): 0.7666
Final F1-macro (val): 0.6744

Classification report (val):

              precision    recall  f1-score   support

    Negativo       0.77      0.90      0.83      2000
      Neutro       0.53      0.25      0.34      1000
    Positivo       0.82      0.89      0.85      2000

    accuracy                           0.77      5000
   macro avg       0.70      0.68      0.67      5000
weighted avg       0.74      0.77      0.74      5000

Labels: ['Negativo', 'Neutro', 'Positivo']

Macro Precision/Recall/F1: 0.7048 0.681 0.6744
Recall Negativo (val): 0.898


## 7) Política productiva: `review_required` por umbral de confianza

**Requisito hackathon (plus):** manejo operacional cuando la confianza es baja.

- Si `max_prob < threshold` → `review_required = True`
- Esto reduce riesgos de falsos negativos en atención al cliente.

Ajusta el umbral según tolerancia al riesgo.

In [10]:
# =========================
# 7) Umbral de revisión
# =========================
THRESHOLD = 0.60

needs_review = val_maxp < THRESHOLD
print("Casos a revisión (VAL):", int(needs_review.sum()), "/", len(needs_review),
      f"({needs_review.mean()*100:.1f}%)")

auto_idx = ~needs_review
print("Auto-coverage:", round(float(auto_idx.mean()), 4))
print("F1-macro (VAL, solo auto):", round(float(f1_score(y_val[auto_idx], val_pred[auto_idx], average='macro')), 4))

Casos a revisión (VAL): 1245 / 5000 (24.9%)
Auto-coverage: 0.751
F1-macro (VAL, solo auto): 0.6913


## 8) Evaluación final (TEST holdout)

Medición realista al final del notebook (como en producción).

In [11]:
X_test = test_es["text_raw"]
y_test = test_es["sentiment"]

test_pred = final_pipe.predict(X_test)
test_proba = final_pipe.predict_proba(X_test)
test_maxp = test_proba.max(axis=1)

print("Accuracy (test):", round(float(accuracy_score(y_test, test_pred)), 4))
print("F1-macro (test):", round(float(f1_score(y_test, test_pred, average='macro')), 4))
print("\nClassification report (test):\n")
print(classification_report(y_test, test_pred))

labels = list(final_pipe.classes_)
cm = confusion_matrix(y_test, test_pred, labels=labels)
pd.DataFrame(cm, index=labels, columns=labels)

Accuracy (test): 0.7754
F1-macro (test): 0.6862

Classification report (test):

              precision    recall  f1-score   support

    Negativo       0.78      0.90      0.84      2000
      Neutro       0.57      0.27      0.37      1000
    Positivo       0.81      0.90      0.85      2000

    accuracy                           0.78      5000
   macro avg       0.72      0.69      0.69      5000
weighted avg       0.75      0.78      0.75      5000



,Negativo,Neutro,Positivo
Negativo,1808,109,83
Neutro,399,270,331
Positivo,104,97,1799


## 9) Explicabilidad ligera (para demo)

Para TF‑IDF + modelo lineal, inspeccionamos términos más influyentes.
Con calibración, el estimador interno se accede vía `final_pipe.named_steps["model"].calibrated_classifiers_`.

> Nota: en clasificación multiclase, LR tiene `coef_` por clase (one-vs-rest).

In [12]:
def top_terms_per_class_from_lr(lr_model: LogisticRegression, vectorizer: TfidfVectorizer, top_k: int = 12):
    feature_names = np.array(vectorizer.get_feature_names_out())
    classes = lr_model.classes_
    coefs = lr_model.coef_
    out = {}
    for i, cls in enumerate(classes):
        top_pos_idx = np.argsort(coefs[i])[-top_k:][::-1]
        top_neg_idx = np.argsort(coefs[i])[:top_k]
        out[str(cls)] = {
            "top_positive": feature_names[top_pos_idx].tolist(),
            "top_negative": feature_names[top_neg_idx].tolist(),
        }
    return out

# Obtener el LR ya entrenado dentro del calibrador
cal = final_pipe.named_steps["model"]
lr_fitted = cal.calibrated_classifiers_[0].estimator  # uno de los folds internos ya estimado

top_terms = top_terms_per_class_from_lr(lr_fitted, final_pipe.named_steps["tfidf"], top_k=12)
top_terms

{'Negativo': {'top_positive': ['no',
   'mala',
   'dos estrellas',
   'no funciona',
   'no cumple',
   'decepcionante',
   'fatal',
   'decepción',
   'mal',
   'decepcionado',
   'roto',
   'se'],
  'top_negative': ['perfecto',
   'buena',
   'buen',
   'genial',
   'perfectamente',
   'bien',
   'perfecta',
   'un poco',
   'excelente',
   'correcto',
   'muy bien',
   'cumple']},
 'Neutro': {'top_positive': ['pero',
   'tres estrellas',
   'regular',
   'aceptable',
   'mejorable',
   'correcto',
   'bien',
   'lo malo',
   'calidad media',
   'normal',
   'esta mal',
   'normalito'],
  'top_negative': ['dos estrellas',
   'recomiendo',
   'encantado',
   'muy',
   'excelente',
   'recomendable',
   'espectacular',
   'perfecto',
   'además',
   'genial',
   'compra',
   'estrella']},
 'Positivo': {'top_positive': ['perfecto',
   'genial',
   'excelente',
   'perfectamente',
   'buena',
   'perfecta',
   'encantado',
   'buen',
   'recomendable',
   'muy bien',
   'encanta',
   'e

## 10) Artefacto productivo: bundle `pipeline + metadata` en `joblib`

Para backend es ideal un **bundle**:
- `pipeline`: limpia + vectoriza + predice + predict_proba
- `meta`: versión, threshold, labels, hash

Esto hace el deploy más robusto.

In [14]:
# =========================
# 10) Serialización productiva (bundle)
# =========================
import pickle

@dataclass
class ModelMeta:
    model_version: str
    threshold: float
    labels: list
    artifact_hash: str

def compute_artifact_hash(obj) -> str:
    blob = pickle.dumps(obj)
    return hashlib.sha256(blob).hexdigest()[:12]

MODEL_VERSION = "sentiment_es_tfidf_lr_calibrated_v1"
artifact_hash = compute_artifact_hash(final_pipe)

meta = ModelMeta(
    model_version=MODEL_VERSION,
    threshold=THRESHOLD,
    labels=list(final_pipe.classes_),
    artifact_hash=artifact_hash
)

bundle = {"pipeline": final_pipe, "meta": meta.__dict__}

MODEL_PATH = "sentiment_bundle.joblib"  # ruta local
joblib.dump(bundle, MODEL_PATH)

print("Guardado:", MODEL_PATH)
print("Meta:", meta)

Guardado: sentiment_bundle.joblib
Meta: ModelMeta(model_version='sentiment_es_tfidf_lr_calibrated_v1', threshold=0.6, labels=['Negativo', 'Neutro', 'Positivo'], artifact_hash='1c13b982c169')


## 11) Funciones listas para Back‑End: validación + predict_one()

Este bloque es lo que BE consume 1:1:
- valida input
- predice
- retorna contrato JSON estable

In [15]:
# =========================
# 11) Contrato de predicción (para backend)
# =========================
def validate_text(text: Any, min_len: int = 5, max_len: int = 2000) -> Optional[Dict[str, str]]:
    if text is None:
        return {"error": "invalid_input", "detail": "text is required"}
    if not isinstance(text, str):
        return {"error": "invalid_input", "detail": "text must be a string"}
    t = text.strip()
    if len(t) < min_len:
        return {"error": "invalid_input", "detail": f"text must have at least {min_len} characters"}
    if len(t) > max_len:
        return {"error": "invalid_input", "detail": f"text must have at most {max_len} characters"}
    return None

def predict_one(text: str, loaded_bundle: Dict[str, Any]) -> Dict[str, Any]:
    err = validate_text(text)
    if err:
        return err

    pipe = loaded_bundle["pipeline"]
    meta = loaded_bundle["meta"]

    proba = pipe.predict_proba([text])[0]
    classes = pipe.classes_
    i = int(np.argmax(proba))
    pred = str(classes[i])
    max_prob = float(proba[i])

    return {
        "prevision": pred,
        "probabilidad": round(max_prob, 4),
        "review_required": bool(max_prob < float(meta["threshold"])),
        "threshold": float(meta["threshold"]),
        "model_version": str(meta["model_version"]),
        "artifact_hash": str(meta["artifact_hash"])
    }

### Smoke test (3 ejemplos: Positivo / Neutro / Negativo)

In [16]:
loaded_bundle = joblib.load(MODEL_PATH)

samples = [
    "El producto es excelente y llegó rápido, muy recomendado.",
    "Está bien, cumple, nada especial.",
    "No funciona, llegó roto y el soporte no responde."
]

for s in samples:
    print(json.dumps(predict_one(s, loaded_bundle), ensure_ascii=False))

{"prevision": "Positivo", "probabilidad": 0.9249, "review_required": false, "threshold": 0.6, "model_version": "sentiment_es_tfidf_lr_calibrated_v1", "artifact_hash": "1c13b982c169"}
{"prevision": "Neutro", "probabilidad": 0.5956, "review_required": true, "threshold": 0.6, "model_version": "sentiment_es_tfidf_lr_calibrated_v1", "artifact_hash": "1c13b982c169"}
{"prevision": "Negativo", "probabilidad": 0.8677, "review_required": false, "threshold": 0.6, "model_version": "sentiment_es_tfidf_lr_calibrated_v1", "artifact_hash": "1c13b982c169"}


## 12) Contrato de API (para Spring Boot)

### Request
```json
POST /sentiment
{ "text": "El producto llegó roto y no funciona" }
```

### Response 200
```json
{
  "prevision": "Negativo",
  "probabilidad": 0.8731,
  "review_required": false,
  "threshold": 0.6,
  "model_version": "sentiment_es_tfidf_lr_calibrated_v1",
  "artifact_hash": "ab12cd34ef56"
}
```

### Response 400 (input inválido)
```json
{ "error": "invalid_input", "detail": "text is required" }
```

## 13) Ejemplos cURL / Postman (3 casos)

```bash
curl -X POST http://localhost:8080/sentiment -H "Content-Type: application/json"   -d '{"text":"El producto es excelente, llegó rápido y funciona perfecto."}'
```

```bash
curl -X POST http://localhost:8080/sentiment -H "Content-Type: application/json"   -d '{"text":"Está bien, cumple lo prometido, nada especial."}'
```

```bash
curl -X POST http://localhost:8080/sentiment -H "Content-Type: application/json"   -d '{"text":"Llegó roto, pésimo servicio y nadie responde."}'
```

## 14) Notas de producción (para jurado)

- **Eficiencia:** TF‑IDF + LR es rápido, CPU‑friendly, y se carga una vez al iniciar la API.
- **Probabilidad confiable:** se usa calibración sigmoid para mejorar interpretación de confianza.
- **Robustez operacional:** `review_required` deriva a revisión humana cuando `probabilidad < threshold`.
- **Reproducibilidad:** semilla fija + metadata con `model_version` y `artifact_hash`.